# Computation of velocity and acceleration 

- [ ] velocity
- [ ] acceleration


In [1]:
import os
from glob import glob

import numpy as np
import pandas as pd
import xarray as xr
import dask.dataframe as dd
import dask.array as da

#import cartopy.crs as ccrs
#import cartopy.feature as cfeature
#import geopandas as gpd
from shapely.geometry import Polygon

%matplotlib inline
from matplotlib import pyplot as plt


import drifters.utils as ut
import pynsitu as pin

from dask.delayed import delayed

/home1/datahome/mdemol/.miniconda3/envs/mdenv/lib/python3.9/site-packages/pyTMD/tools.py:55: UserWarning: ipyleaflet not available
  warnings.warn("ipyleaflet not available")
/home1/datahome/mdemol/.miniconda3/envs/mdenv/lib/python3.9/site-packages/pyTMD/tools.py:56: UserWarning: Some functions will throw an exception if called
  warnings.warn("Some functions will throw an exception if called")


In [2]:
from dask.distributed import Client

if True:
    from dask_jobqueue import PBSCluster
    from dask.distributed import Client
    cluster = PBSCluster(cores=4, processes=4, walltime='04:00:00')
    w = cluster.scale(jobs=5)
else:
    from dask.distributed import LocalCluster
    cluster = LocalCluster()
    
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: http://10.148.0.8:8787/status,
Dashboard: http://10.148.0.8:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.148.0.8:36267,Workers: 0
Dashboard: http://10.148.0.8:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [3]:
# load data

col_sel = ['time', 'id','lon', 'lat', 've', 'vn', 'typebuoy', 'gap',
           'deploy_date', 'deploy_lat','deploy_lon', 'end_date', 'end_lat', 'end_lon', 
           'drogue_lost_date','typedeath', 'lon360', 'err_lat',
           'err_lon', 'err_ve', 'err_vn']

bandwidth = 0.1

df_argos = ut.load_gdp("argos", suffix=f"_filtered_b{bandwidth}")
df_argos = df_argos.where(df_argos!=-1.e+34).reset_index().rename(columns=dict(index="id"))[col_sel].persist()

df_gps = ut.load_gdp("gps", suffix=f"_filtered_b{bandwidth}")
df_gps = (df_gps.where(df_gps!=-1.e+34).reset_index().rename(columns=dict(index="id")))[col_sel].persist() # ATTENTION : replace missing value -1.e+34 by Nan

In [4]:
df_gps

,time,id,lon,lat,ve,vn,typebuoy,gap,deploy_date,deploy_lat,deploy_lon,end_date,end_lat,end_lon,drogue_lost_date,typedeath,lon360,err_lat,err_lon,err_ve,err_vn
npartitions=176,,,,,,,,,,,,,,,,,,,,,
,datetime64[ns],int64,float32,float32,float32,float32,object,float64,datetime64[ns],float64,float64,datetime64[ns],float64,float64,datetime64[ns],float64,float32,float32,float32,float32,float32
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [4]:
# dev
#df_gps = dd.concat([df_gps.get_partition(n) for n in range(3)]).persist()
df = df_gps.get_partition(0).groupby('id').get_group(131969).compute()

In [5]:
df

,time,id,lon,lat,ve,vn,typebuoy,gap,deploy_date,deploy_lat,...,end_date,end_lat,end_lon,drogue_lost_date,typedeath,lon360,err_lat,err_lon,err_ve,err_vn
0,2014-02-18 20:00:00,131969,0.72649,38.723270,-0.0298,-0.3110,b'SVP ',3629.0,2014-02-18,38.793,...,2014-09-12,38.81,1.33,2014-04-25,1.0,0.72649,0.00048,0.00062,0.0182,0.0182
1,2014-02-18 21:00:00,131969,0.72519,38.713181,-0.0320,-0.3121,b'SVP ',3629.0,2014-02-18,38.793,...,2014-09-12,38.81,1.33,2014-04-25,1.0,0.72519,0.00076,0.00098,0.0181,0.0181
2,2014-02-18 22:00:00,131969,0.72413,38.705109,-0.0204,-0.2998,b'SVP ',17971.0,2014-02-18,38.793,...,2014-09-12,38.81,1.33,2014-04-25,1.0,0.72413,0.00064,0.00081,0.0150,0.0150
3,2014-02-18 23:00:00,131969,0.72308,38.697090,-0.0084,-0.2873,b'SVP ',17971.0,2014-02-18,38.793,...,2014-09-12,38.81,1.33,2014-04-25,1.0,0.72308,0.00052,0.00066,0.0117,0.0117
4,2014-02-19 00:00:00,131969,0.72203,38.689072,0.0035,-0.2747,b'SVP ',17971.0,2014-02-18,38.793,...,2014-09-12,38.81,1.33,2014-04-25,1.0,0.72203,0.00044,0.00057,0.0091,0.0091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4953,2014-09-13 05:00:00,131969,1.51884,38.679611,0.0000,0.0016,b'SVP ',3629.0,2014-02-18,38.793,...,2014-09-12,38.81,1.33,2014-04-25,1.0,1.51884,0.00037,0.00047,0.0137,0.0137
4954,2014-09-13 06:00:00,131969,1.51889,38.679630,0.0012,-0.0016,b'SVP ',0.0,2014-02-18,38.793,...,2014-09-12,38.81,1.33,2014-04-25,1.0,1.51889,0.00050,0.00064,0.0184,0.0184
4955,2014-09-13 07:00:00,131969,1.51894,38.679630,0.0024,0.0016,b'SVP ',3629.0,2014-02-18,38.793,...,2014-09-12,38.81,1.33,2014-04-25,1.0,1.51894,0.00037,0.00047,0.0137,0.0137
4956,2014-09-13 08:00:00,131969,1.51902,38.679680,0.0012,0.0017,b'SVP ',3629.0,2014-02-18,38.793,...,2014-09-12,38.81,1.33,2014-04-25,1.0,1.51902,0.00055,0.00071,0.0185,0.0185


## Computation : 
Derivation of velocities and acceleration :
- ve, vn already computed via LOWESS METHOD
- vex, vey centered derivation using projection of lon, lat
- aex, any double centered derivation using projection of lon, lat
- ae, an centered derivation of ve, vn

In [5]:
def c_vel_acc(df):
    _geo = pin.geo.GeoAccessor(df)
    df0 = _geo.compute_velocities(time="time", centered=True, v_name={'ve':'vex', 'vn':'vny', 'v':'vxy'})
    print(df0.columns)
    df1 = _geo.compute_acceleration(time='time', acc_name={'ae':'ae', 'an':'an', 'a':'aen'}, from_ = ('vevn','ve','vn'), centered_velocity=True, )
    print(df1.columns)
    df2 = _geo.compute_acceleration(time='time', acc_name={'ae':'aex', 'an':'any', 'a':'axy'}, from_ = ('lonlat','lon','lat') )
    print(df2.columns)
    #return df0, df1, df2
    return df0.merge(df1, how='outer', on=['time', 'id']).merge(df2, how='inner', on=['time', 'id', 'lon', 'lat']).merge(df[col_sel], how='inner', on=['time', 'id', 'lon', 'lat', 've', 'vn'])

In [22]:
df_va = c_vel_acc(df)
df_va

Index(['time', 'id', 'lon', 'lat', 'vex', 'vny', 'vxy'], dtype='object')
Index(['time', 'id', 've', 'vn', 'ae', 'an', 'aen'], dtype='object')
Index(['time', 'id', 'lon', 'lat', 'aex', 'any', 'axy'], dtype='object')


,time,id,lon,lat,vex,vny,vxy,ve,vn,ae,an,aen,aex,any,axy
0,2014-02-18 20:00:00,131969,0.72649,38.723270,-0.028510,-0.419881,0.420848,-0.0298,-0.3110,1.305555e-06,1.555553e-06,2.030818e-06,1.610036e-06,1.728524e-05,1.736006e-05
1,2014-02-18 21:00:00,131969,0.72519,38.713181,-0.028510,-0.419881,0.420848,-0.0320,-0.3121,1.305555e-06,1.555553e-06,2.030818e-06,1.610036e-06,1.728524e-05,1.736006e-05
2,2014-02-18 22:00:00,131969,0.72413,38.705109,-0.025493,-0.359729,0.360631,-0.0204,-0.2998,3.277778e-06,3.444445e-06,4.754790e-06,6.642208e-08,4.575454e-07,4.623415e-07
3,2014-02-18 23:00:00,131969,0.72308,38.697090,-0.025375,-0.358201,0.359099,-0.0084,-0.2873,3.319445e-06,3.486114e-06,4.813700e-06,-7.868580e-10,9.443854e-11,7.925050e-10
4,2014-02-19 00:00:00,131969,0.72203,38.689072,-0.025378,-0.358258,0.359156,0.0035,-0.2747,3.319444e-06,3.499997e-06,4.823763e-06,-7.870937e-10,-3.258051e-08,3.259002e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4953,2014-09-13 05:00:00,131969,1.51884,38.679611,0.000363,0.001477,0.001521,0.0000,0.0016,1.805556e-07,-4.444444e-07,4.797199e-07,4.706304e-07,-9.802473e-08,4.807306e-07
4954,2014-09-13 06:00:00,131969,1.51889,38.679630,0.001209,-0.000017,0.001209,0.0012,-0.0016,3.333333e-07,0.000000e+00,3.333333e-07,-8.003918e-10,-1.633747e-07,1.633766e-07
4955,2014-09-13 07:00:00,131969,1.51894,38.679630,0.001570,0.000161,0.001579,0.0024,0.0016,0.000000e+00,4.583333e-07,4.583333e-07,2.016984e-07,4.247743e-07,4.702291e-07
4956,2014-09-13 08:00:00,131969,1.51902,38.679680,0.001933,0.000857,0.002115,0.0012,0.0017,0.000000e+00,-2.222222e-07,2.222222e-07,-1.425016e-13,-2.613995e-07,2.613995e-07


In [6]:
df_gps_out = df_gps.groupby("id").apply(c_vel_acc).set_index('time')
df_argos_out = df_argos.groupby("id").apply(c_vel_acc).set_index('time')

Index(['time', 'id', 'lon', 'lat', 'vex', 'vny', 'vxy'], dtype='object')
Index(['time', 'id', 've', 'vn', 'ae', 'an', 'aen'], dtype='object')
Index(['time', 'id', 'lon', 'lat', 'aex', 'any', 'axy'], dtype='object')
Index(['time', 'id', 'lon', 'lat', 'vex', 'vny', 'vxy'], dtype='object')
Index(['time', 'id', 've', 'vn', 'ae', 'an', 'aen'], dtype='object')
Index(['time', 'id', 'lon', 'lat', 'aex', 'any', 'axy'], dtype='object')


# Store in parquet

In [7]:
root_dir = "/home1/datawork/mdemol/GDP"
parquet_argos = os.path.join(root_dir, "argos_av_time.parquet")
parquet_gps = os.path.join(root_dir, "gps_av_time.parquet")

In [10]:
df_load_gps = (df_gps_out.repartition(partition_size="100MB").persist())
df_load_argos = (df_argos_out.repartition(partition_size="100MB").persist())

In [11]:
df_load_gps.to_parquet(parquet_gps, engine='pyarrow') # alternative: fastparquet pyarrow
df_load_argos.to_parquet(parquet_argos, engine='pyarrow') # alternative: fastparquet

In [62]:
df_load_gps

,id,lon,lat,vex,vny,vxy,ve,vn,ae,an,aen,aex,any,axy
npartitions=176,,,,,,,,,,,,,,
2001-07-01 02:00:00,int64,float32,float32,float64,float64,float64,float32,float32,float64,float64,float64,float64,float64,float64
2008-08-15 17:00:00,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-25 20:00:00,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-04-06 03:00:00,...,...,...,...,...,...,...,...,...,...,...,...,...,...


# Open test

In [12]:
df_gps_reload = dd.read_parquet(parquet_gps).persist()

In [13]:
df_gps_reload['ve'].compute()

time
2001-07-01 02:00:00    0.6460
2001-07-01 03:00:00    0.6466
2001-07-01 04:00:00    0.6440
2001-07-01 05:00:00    0.6368
2001-07-01 06:00:00    0.6296
                        ...  
2020-04-06 03:00:00    0.0721
2020-04-06 03:00:00   -0.0199
2020-04-06 03:00:00    0.0318
2020-04-06 03:00:00    0.1766
2020-04-06 03:00:00    0.0386
Name: ve, Length: 40529116, dtype: float32

In [14]:
df_gps_reload.columns

Index(['id', 'lon', 'lat', 'vex', 'vny', 'vxy', 've', 'vn', 'ae', 'an', 'aen',
       'aex', 'any', 'axy', 'typebuoy', 'gap', 'deploy_date', 'deploy_lat',
       'deploy_lon', 'end_date', 'end_lat', 'end_lon', 'drogue_lost_date',
       'typedeath', 'lon360', 'err_lat', 'err_lon', 'err_ve', 'err_vn'],
      dtype='object')

In [15]:
cluster.close()

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
